In [1]:
import matplotlib.pyplot as plt

import numpy as np
import networkx as nx

import pandas as pd
import math

from scipy.integrate import odeint

# Information Distance

Application of information distance to the WAN as an effective spreading distance for RW on the graph.
The probability of a walker to be in any point in WAN, starting from node $i$, $\mathbf{p}(t \mid i)$ embeds information of multiple paths via successive applications of the Laplacian operator. The
idea that low probabilities $p_{k}(t \mid i)$ are associated with large distances can be embedded by taking the negative of the logarithm of the probability, in analogy with Shannon's entropy:


$$ID_{t}(s \rightarrow k) = - \log p_{k}\left(t \mid s \right)$$


 $p_{k}\left(t \mid s \right)$ represents the $k-th$ entry associated with node $k$ of the probability state $ \mathbf{p}(t \mid s) = \mathbf{v_{s}}\cdot e^{-t L^{RW}}$, where $\mathbf{v_{s}}$ is the probability vector at time $t_{0} = 0$ (with $s-th$ entry equal to 1 and other entries equal to 0). Similarly to diffusion distance, we shall find the best timescale $t^{*}$, which in this case results in $t^{*} = 1$.


# Load graphs

Load datasets of nodes and networkx object edgelist

Note:
- Requires graph object data for WAN previously symmetrized and generated via `WAN_graph_generation.ipynb` notebook
- Mapping between airports and integers defined for indexing the airports in the arrays and matrices used in information distance numerical solutions.
- Some nodes are disconnected from the WAN largest connected component, these have an `information distance == np.inf` 


In [2]:
# Load aggregated and check that it is ok with the original

# Keys for pd dataframe
nodes_keys = ['node_id','iata_code','airport_name','iso_a3','lat','lon']
links_keys = ['source','target','flights','seats']

# PATH where edgelist is stored
period = '2020-11' #'YYYY-MM'
PATH_edgelist = "./wan_graphs/G_WAN_{}.edgelist".format(period)

# WAN dataset
WAN = pd.read_csv("./wan/wan_nodes.csv")
WAN_edges = pd.read_csv("./wan/wan_link_{}.csv".format(period))

# Edgelist
G_WAN = nx.read_edgelist(PATH_edgelist, data = True)
airports = list(G_WAN.nodes)

# Set airports unique mapping (from node label to index)
airports_ordering = {}
for index,value in enumerate(airports):
    airports_ordering[value] = index
    
# Check if G is connected
print("Is connected:",nx.is_connected(G_WAN))

False


## Functions to compute ID

In [3]:
# Functions

def RW(x,t,L_norm):
    '''
    Compute derivative of x at time t for random walker.
    Args:
        x:np.ndarray 1-dim
        t:np.ndarray (times)
        L_norm:np.ndarray 2-dim (RW-normalized graph Laplacian)
    Returns:
        dxdt:np.ndarray      
    '''
    dxdt = - L_norm.dot(x)
    return dxdt

def Full_Probability_State(L_norm, times, node):
    '''
    Get final probability state for RW starting at source [node]
    Args:
        L_norm:np.ndarray 2-dim (RW-normalized graph Laplacian)
        times:np.ndarray 1-dim (set of times odenint() computes explicitly the solution of the ODE)
        node:int (node_index of the source node)
    Returns:
        x_out:np.ndarray 2-dim (solution of the ODE for each time in times and each node in the graph
                                eg: x_out[t][node_index] is prob of RW to be found at node_index at time t)
    '''

    state = np.zeros(len(L_norm[0]))
    state[node] = 1
    x_out = odeint(RW, state, times, args = (L_norm,))

    return x_out

# -----

def Information_Distance(G, times, airports_ordering, source):
    
    '''
    Information distance implementation for WAN network of airports
    Args:
        G:networkx graph object
        times:np.ndarray 1-dim (set of times odenint() computes explicitly the solution of the ODE)
        airports_ordering: dict (mapping from node label to unique integer for indexing in arrays)
        source:str (node label of the source node, currently working with WAN nodes indexed via 'lid')
    Returns:
        Distances:np.ndarray dict (dict of distances from source node, where Distances[lid]
                                is the distance of airport lid having index i = airports_ordering[lid]. 
                                note that: Distances[source] = 0)
    '''
    num_nodes = G.number_of_nodes()
    airports = list(G.nodes)
    
    # Fix airports as nodelist to be preserved via indexing
    Adj = nx.adjacency_matrix(G, nodelist=airports, weight = 'weight')
    
    # Nodes degrees
    degrees = np.zeros(num_nodes)
    for (n, d) in G.degree(weight = 'weight'):
        degrees[airports_ordering[n]] = np.float(d)

    # Compute Normalized RW Laplacian
    I = np.zeros(shape = (num_nodes,num_nodes))
    D_inv = np.zeros(shape = (num_nodes,num_nodes))
    
    for lid in airports:
        i = airports_ordering[lid]
        degree = degrees[i]
        
        I[i][i] = 1
        D_inv[i][i] = 1./degree
    L_norm = I - D_inv@Adj     
    
    # Compute probability state 
    Probability_State = Full_Probability_State(L_norm, times, airports_ordering[source])[-1]
    Probability_State = Probability_State/max(Probability_State)

    # Distances array
    Distances = {}

    for lid in airports:
                
        id_lid = airports_ordering[lid]
        p = Probability_State[id_lid]
        
        Distances[lid] = -np.log10(p)
    
    return Distances


## Information Distance

Test from source node LHR (London Heathrow)

In [4]:
# Set source node and times for the solution of ODE
source = 'LHR'

# End time (t = 1) is the value used for the ID
end_time = 1.0
times = np.arange(0,end_time,0.1)

# Compute Information Distance
Distances_source = Information_Distance(G_WAN,times, airports_ordering, source)


/var/folders/ms/3n67wbb95xn26d6lg9q5sjw40000gn/T/ipykernel_3874/1003882458.py:59: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  degrees[airports_ordering[n]] = np.float(d)
/var/folders/ms/3n67wbb95xn26d6lg9q5sjw40000gn/T/ipykernel_3874/1003882458.py:85: RuntimeWarning: divide by zero encountered in log10
  Distances[lid] = -np.log10(p)


In [5]:

# Remove inf elements (from nodes in the disconnected network (G_WAN is not connected))
Distances_source = {key:val for key, val in Distances_source.items() if val != np.inf}

# Sort distances
sorted_dict = dict(sorted(Distances_source.items(), key=lambda item: item[1]))

# Print targets
for key in list(sorted_dict.keys()):
    print(key, np.round(sorted_dict[key],4), 
          list(WAN[WAN['iata_code']==key]['airport_name'].values)[0],
            list(WAN[WAN['iata_code']==key]['iso_a3'].values)[0])



LHR -0.0 London Heathrow Apt GBR
GIB 0.1902 Gibraltar GIB
NQY 0.3865 Newquay GBR
BHD 0.3957 Belfast George Best City Apt GBR
ORK 0.5096 Cork IRL
JER 0.6249 Jersey GBR
ABZ 0.6522 Aberdeen GBR
KEF 0.6682 Reykjavik Keflavik International Apt ISL
MME 0.6798 Durham GBR
INV 0.742 Inverness GBR
NCL 0.7864 Newcastle GBR
GLA 0.8309 Glasgow International Airport GBR
EDI 0.8575 Edinburgh GBR
BGI 0.9288 Barbados BRB
TXL 1.0807 Berlin Tegel Apt DEU
CFU 1.083 Kerkyra GRC
LIN 1.0973 Milan Linate Apt ITA
LOS 1.1046 Lagos NGA
LSI 1.1322 Shetland Islands Sumburgh Apt GBR
AUH 1.1522 Abu Dhabi International Apt ARE
EXT 1.1538 Exeter GBR
KOI 1.1601 Kirkwall GBR
ZYL 1.1621 Sylhet BGD
BAH 1.1693 Bahrain BHR
LCA 1.1924 Larnaca CYP
BRR 1.193 Barra GBR
MAN 1.2047 Manchester GBR
HEL 1.221 Helsinki-Vantaa FIN
DBV 1.2277 Dubrovnik HRV
DUB 1.2299 Dublin IRL
GVA 1.2324 Geneva International CHE
TRE 1.2326 Tiree GBR
SYY 1.2431 Stornoway GBR
SOU 1.245 Southampton GBR
HUY 1.2539 Humberside GBR
CMB 1.2637 Bandaranaike In

GDN 2.0264 Gdansk Lech Walesa POL
MMX 2.0268 Malmo Airport SWE
AAR 2.0277 Aarhus Tirstrup Airport DNK
RKT 2.0301 Ras al Khaimah ARE
EBL 2.0307 Erbil IRQ
ALG 2.031 Algiers DZA
AXU 2.0333 Axum ETH
EBB 2.0341 Entebbe UGA
BTS 2.0347 Bratislava SVK
LEI 2.0361 Almeria ESP
SMV 2.0362 St Moritz CHE
GRZ 2.0362 Graz AUT
JKL 2.0365 Kalymnos Island GRC
PRN 2.0403 Pristina SRB
KGL 2.041 Kigali RWA
EDL 2.0431 Eldoret KEN
LPI 2.044 Linkoping SWE
RAK 2.0443 Marrakech MAR
AQJ 2.0444 Aqaba JOR
MNL 2.0471 Manila Ninoy Aquino International Apt PHL
FNA 2.0473 Freetown Lungi International Apt SLE
LPA 2.0475 Gran Canaria ESP
TUF 2.0476 Tours Val de Loire Airport FRA
JIJ 2.049 Jijiga ETH
HYD 2.0507 Hyderabad Rajiv Gandhi Intl Arpt IND
YAM 2.0511 Sault Ste Marie CAN
HRG 2.0535 Hurghada EGY
ROB 2.0576 Monrovia Roberts International Apt LBR
CIA 2.0577 Rome Ciampino Apt ITA
MQX 2.0579 Mekele ETH
CGN 2.0598 Cologne/Bonn Apt DEU
LFW 2.0619 Lome TGO
HAJ 2.0631 Hannover DEU
SID 2.0642 Sal Island CPV
SNN 2.067 Shannon

HOR 2.326 Horta PRT
GLK 2.3267 Galcaio SOM
GZP 2.3277 Gazipasa TUR
FKB 2.3279 Karlsruhe/Baden-Baden Baden Airpark DEU
ATL 2.331 Atlanta Hartsfield-Jackson Intl Apt USA
ANR 2.3314 Antwerp BEL
PSR 2.3315 Pescara ITA
AGA 2.333 Agadir MAR
CJB 2.3331 Coimbatore IND
SAL 2.3332 San Salvador SLV
SXR 2.3351 Srinagar IND
BBI 2.3356 Bhubaneshwar IND
KCM 2.3364 Kahramanmaras TUR
BAY 2.3401 Baia Mare ROU
OMR 2.3401 Oradea ROU
RPR 2.3407 Raipur IND
YVP 2.3415 Kuujjuaq CAN
HLA 2.3436 Johannesburg Lanseria International Apt ZAF
TFN 2.3453 Tenerife Norte ESP
IXC 2.3459 Chandigarh IND
KUA 2.3501 Kuantan MYS
ORY 2.3502 Paris Orly Apt FRA
IXJ 2.351 Jammu IND
FSP 2.3532 St Pierre SPM
OBN 2.3534 Oban GBR
NRN 2.3544 Duesseldorf Weeze Airport DEU
CUF 2.3548 Cuneo ITA
IEV 2.3548 Kyiv Zhuliany Intl Apt UKR
EUN 2.3559 Laayoune MAR
IXX 2.3577 Bidar IND
GBI 2.3577 Gulbarga IND
XFW 2.362 Hamburg Finkenwerder DEU
SXF 2.3632 Berlin Schoenefeld Apt DEU
BIA 2.3649 Bastia FRA
KTM 2.365 Kathmandu NPL
FEZ 2.3653 Fes MAR
Y

JJM 2.5864 Meru-Kinna KEN
GGM 2.5866 Kakamega KEN
HAD 2.5874 Halmstad SWE
BUF 2.5912 Buffalo USA
CDT 2.5947 Castellon de la Plana ESP
GSO 2.5949 Greensboro/High Point USA
NOS 2.5949 Nosy-Be MDG
KME 2.5956 Cyangugu RWA
JAX 2.596 Jacksonville International Apt USA
SUX 2.5962 Sioux City USA
YHS 2.5966 Sechelt CAN
GDX 2.5969 Magadan RUS
DIB 2.5983 Dibrugarh IND
DAY 2.5995 Dayton James M. Cox Intl Apt USA
KSY 2.6012 Kars TUR
IJK 2.6017 Izhevsk RUS
VVO 2.6017 Vladivostok RUS
KUT 2.6018 Kutaisi GEO
KDH 2.6018 Kandahar AFG
TLH 2.6018 Tallahassee USA
GOM 2.6019 Goma COD
TVC 2.6022 Traverse City USA
ABK 2.6023 Kebri Dehar ETH
KJT 2.6031 Majalengka IDN
HAS 2.6032 Hail SAU
VOG 2.6041 Volgograd RUS
HAH 2.6044 Moroni Prince Said Ibrahim COM
ABI 2.6057 Abilene Regional Apt USA
OGU 2.6061 Ordu Giresun TUR
LFM 2.6066 Lamerd IRN
GRK 2.6081 Killeen/Fort Hood Regional/R. Gray AAF USA
PBI 2.609 West Palm Beach International Apt USA
GDE 2.609 Gode ETH
BHI 2.6103 Bahia Blanca ARG
ASU 2.6104 Asuncion PRY
TLM 

LED 2.8035 St Petersburg Pulkovo Apt RUS
SHL 2.8042 Shillong IND
DMU 2.8042 Dimapur IND
ZIG 2.8046 Ziguinchor SEN
YUM 2.805 Yuma International Apt USA
UEL 2.8052 Quelimane MOZ
CYO 2.806 Cayo Largo del Sur CUB
KKJ 2.8076 Kitakyushu JPN
DTW 2.8078 Detroit Metropolitan Wayne County USA
BSG 2.8085 Bata GNQ
VCT 2.8095 Victoria USA
ALB 2.8099 Albany International Airport USA
ICT 2.8102 Wichita Dwight D. Eisenhower Apt USA
INI 2.8111 Nis SRB
LPK 2.8115 Lipetsk RUS
CAC 2.8124 Cascavel BRA
BDL 2.8159 Hartford Bradley International Apt USA
RAH 2.8161 Rafha SAU
WAE 2.8161 Wadi al Dawaser SAU
TUI 2.8161 Turaif SAU
AZO 2.8169 Kalamazoo/Battle Creek USA
GRI 2.8177 Grand Island USA
CBT 2.8182 Catumbela AGO
BCA 2.8188 Baracoa CUB
GAO 2.8188 Guantanamo Bay NS CUB
GER 2.8188 Nueva Gerona CUB
SVX 2.8193 Yekaterinburg RUS
DLH 2.8199 Duluth USA
OAJ 2.8204 Jacksonville USA
GSV 2.8205 Saratov RUS
FAY 2.8206 Fayetteville USA
MYR 2.8213 Myrtle Beach International USA
STS 2.8219 Santa Rosa USA
BTU 2.823 Bintulu

...

BZK 3.3402 Bryansk RUS
PIE 3.3426 Tampa St Pete-Clearwater Intl Apt USA
YTE 3.3439 Cape Dorset CAN
SEU 3.344 Seronera TZA
GHB 3.3447 Governors Harbour BHS
UKB 3.3448 Osaka Kobe Airport JPN
EPA 3.3492 El Palomar ARG
SFB 3.3512 Orlando Sanford International Airport USA
DYR 3.3544 Anadyr RUS
BUT 3.3552 Bathpalathang BTN
IQQ 3.3557 Iquique CHL
YSQ 3.3592 Songyuan CHN
CPD 3.3598 Coober Pedy AUS
OLP 3.3598 Olympic Dam AUS
WYA 3.3598 Whyalla AUS
CED 3.3598 Ceduna AUS
PLO 3.3598 Port Lincoln AUS
KGC 3.3598 Kingscote AUS
YRS 3.3607 Red Sucker Lake CAN
NTL 3.3614 Newcastle Williamtown Apt AUS
CHC 3.3619 Christchurch NZL
SEB 3.3628 Sebha LBY
AKF 3.3628 Kufra LBY
ZIS 3.3628 Zintan LBY
URT 3.3652 Surat Thani THA
YAP 3.3674 Yap FSM
TBW 3.3689 Tambov RUS
IWA 3.3692 Ivanovo RUS
CJC 3.3723 Calama CHL
NNM 3.3742 Naryan-Mar RUS
ANF 3.3746 Antofagasta CHL
PLN 3.3756 Pellston USA
ESC 3.3756 Escanaba USA
APN 3.3756 Alpena USA
BGM 3.3756 Binghamton USA
CPX 3.3756 Culebra PRI
JEG 3.3768 Aasiaat GRL
CJS 3.3806

HGU 3.6087 Mount Hagen PNG
VDH 3.6094 Dong Hoi VNM
BAV 3.6107 Baotou CHN
DQA 3.612 Daqing CHN
RFP 3.6121 Raiatea Island PYF
ELM 3.6125 Elmira/Corning USA
TDK 3.614 Taldykorgan KAZ
TPP 3.615 Tarapoto PER
CCA 3.6157 Chimore BOL
PXU 3.616 Pleiku VNM
UIH 3.6188 Quinhon VNM
KHK 3.6216 Khark Island IRN
YVZ 3.6219 Deer Lake CAN
BAR 3.6227 Qionghai CHN
VLN 3.6256 Valencia VEN
VCL 3.6261 Tamky VNM
KGP 3.6264 Kogalym RUS
IGT 3.6264 Magas RUS
KRO 3.6264 Kurgan RUS
KLV 3.6264 Karlovy Vary CZE
TSF 3.6264 Venice Treviso/Sant'Angelo Apt ITA
NVI 3.6264 Navoi UZB
AZN 3.6264 Andizhan UZB
CZS 3.6282 Cruzeiro Do Sul BRA
TNA 3.6282 Jinan CHN
AXR 3.6291 Arutua PYF
CIZ 3.6294 Coari BRA
HVD 3.6305 Khovd MNG
LWY 3.6313 Lawas MYS
RIZ 3.6314 Rizhao CHN
HFT 3.6325 Hammerfest NOR
SHE 3.6332 Shenyang CHN
MMJ 3.6351 Matsumoto JPN
BTJ 3.637 Banda Aceh IDN
IRM 3.6374 Igrim RUS
TUO 3.6395 Taupo NZL
BRA 3.6396 Barreiras BRA
TKK 3.6404 Chuuk FSM
HDG 3.6405 Handan CHN
ITB 3.6422 Itaituba BRA
DDG 3.6429 Dandong CHN
VKG 3.6

YFJ 3.9017 Snare Lake CAN
YRA 3.9017 Rae Lakes CAN
YLE 3.9017 Whati CAN
YSG 3.9017 Lutselke Snowdrift CAN
YFS 3.9017 Fort Simpson CAN
HKB 3.9018 Healy Lake USA
GST 3.9047 Gustavus Airport USA
EXI 3.9047 Excursion Inlet USA
HNH 3.9047 Hoonah USA
JIQ 3.9083 Qianjiang CHN
PGU 3.9113 Asaloyeh IRN
FLZ 3.912 Sibolga IDN
BVS 3.9141 Breves BRA
RBY 3.915 Ruby USA
ROO 3.9155 Rondonopolis BRA
YBT 3.9166 Brochet CAN
CHT 3.9173 Chatham Island NZL
WSK 3.9177 Wushan CHN
GGB 3.9188 Agua Boa BRA
FUG 3.9197 Fuyang CHN
HAK 3.9206 Haikou CHN
FYU 3.9216 Fort Yukon USA
ACX 3.9226 Xingyi CHN
UYN 3.9232 Yulin CHN
ZAT 3.9235 Zhaotong CHN
TXN 3.9239 Huangshan CHN
SIT 3.9241 Sitka USA
CBI 3.9251 Cape Barren Island AUS
BHE 3.9257 Blenheim NZL
DLU 3.9277 Dali CHN
IQN 3.9277 Qingyang CHN
DPS 3.9282 Denpasar-Bali IDN
SRK 3.9286 Siorapaluk GRL
ZFD 3.93 Fond du Lac CAN
UKG 3.9302 Ust-Kuyga RUS
ZKP 3.9302 Zyryanka RUS
CKH 3.9302 Chokurdakh RUS
IKS 3.9302 Tiksi RUS
MQJ 3.9302 Khonuu RUS
VHV 3.9302 Verkhnevilyuysk RUS
BQ

KHZ 4.3864 Kauehi PYF
YER 4.3875 Fort Severn CAN
SWQ 4.3929 Sumbawa IDN
PAC 4.3962 Panama City Marcos A.Gelabert Intl PAN
HPG 4.3994 Shennongjia CHN
LXA 4.4019 Lhasa/Lasa CHN
GSM 4.4027 Gheshm Island IRN
PUK 4.4034 Pukarua PYF
PLW 4.4037 Palu IDN
HGD 4.4049 Hughenden AUS
JGD 4.4103 Jiagedaqi CHN
ULP 4.4119 Quilpie AUS
DBC 4.4138 Baicheng CHN
HMI 4.4209 Hami CHN
MDU 4.4235 Mendi PNG
KIH 4.4271 Kish Island IRN
YRB 4.4291 Resolute CAN
GOQ 4.4295 Golmud CHN
GYU 4.4296 Guyuan CHN
CYI 4.4314 Chiayi TWN
REA 4.4318 Reao PYF
BPX 4.4347 Qamdo/Changdu CHN
CAJ 4.4366 Canaima VEN
KOE 4.4382 Kupang IDN
HBQ 4.4406 Qilian CHN
HXD 4.4406 Delingha CHN
AIT 4.4435 Aitutaki Island COK
YUS 4.4469 Yushu CHN
TNN 4.4476 Tainan TWN
GXH 4.458 Xiahe CHN
GMQ 4.4656 Golog CHN
PIZ 4.4671 Point Lay USA
SOQ 4.4717 Sorong IDN
OLH 4.474 Old Harbor USA
KLN 4.474 Larsen Bay USA
AKK 4.474 Akhiok USA
KYK 4.474 Karluk USA
JGN 4.4745 Jiayuguan CHN
HGN 4.4746 Mae Hong Son THA
PHS 4.4746 Phitsanulok THA
TST 4.4746 Trang THA
PRH

RBV 5.2173 Ramata Island SLB
GZO 5.2208 Gizo SLB
UMD 5.2227 Uummannaq GRL
PRS 5.2266 Parasi SLB
FKQ 5.231 Fak Fak IDN
YHO 5.2474 Hopedale CAN
KKA 5.2508 Koyuk USA
WBB 5.2508 Stebbins USA
RCM 5.2522 Richmond AUS
YPC 5.2547 Paulatuk CAN
ZFM 5.2547 Fort McPherson CAN
LAK 5.2547 Aklavik CAN
KNG 5.2573 Utarom IDN
ONG 5.2606 Mornington Island AUS
KSM 5.2661 St Mary's USA
NUP 5.2735 Nunapitchuk USA
WWT 5.2741 Newtok USA
CHY 5.2742 Choiseul Bay SLB
RKV 5.2773 Reykjavik Apt ISL
VLI 5.2796 Port Vila VUT
ATT 5.2813 Atmautluak USA
RUS 5.2987 Marau SLB
NBX 5.3026 Nabire IDN
DOB 5.3037 Dobo IDN
YZG 5.3106 Salluit CAN
IQM 5.3239 Qiemo CHN
FHZ 5.3257 Fakahina PYF
GIC 5.3305 Boigu Island AUS
KKH 5.3314 Kongiganak USA
EJN 5.3429 Ejina Banner CHN
RHT 5.3429 Alxa Right Banner CHN
KOT 5.3873 Kotlik USA
SFD 5.3914 San Fernando Apure VEN
NME 5.3944 Nightmute USA
MNK 5.4076 Maiana KIR
AEA 5.4076 Abemama KIR
NON 5.4076 Nonouti KIR
OKR 5.4178 Yorke Island AUS
PQS 5.418 Pilot Station USA
RSH 5.4313 Russian Missi